# Amazon Athena - Convert TSV Data To Parquet

In this notebook, we will show you how you can easily convert that data now into Apache Parquet file format.

<img src="img/c3-08.png" width="90%" align="left">

In [1]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

# Set S3 prefixes
parquet_prefix = 'amazon-reviews-pds/parquet'

# Set S3 path to Parquet data
s3_path_parquet = 's3://{}/{}'.format(bucket, parquet_prefix)

# Set Athena parameters
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'
table_name_parquet = 'amazon_reviews_parquet'

### Install PyAthena

In [2]:
!pip install -q PyAthena==1.8.0

In [3]:
from pyathena import connect
from pyathena.pandas_cursor import PandasCursor
from pyathena.util import as_pandas

### Create Parquet Files from TSV Table

As you can see from the query below, we’re also adding a new `year` column to our dataset by converting the `review_date` string to a date format, and then cast the year out of the date. Let’s store the year value as an integer. And let's partition the Parquet data by `Product Category`.

In [4]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = 's3://{0}/athena/staging'.format(bucket)

In [5]:
# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM {}.{}""".format(database_name, table_name_parquet, s3_path_parquet, database_name, table_name_tsv)

print(statement)

CREATE TABLE IF NOT EXISTS dsoaws.amazon_reviews_parquet
WITH (format = 'PARQUET', external_location = 's3://sagemaker-us-west-2-086401037028/amazon-reviews-pds/parquet', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM dsoaws.amazon_reviews_tsv


### Execute statement using connection cursor
This can take a few minutes.  Please be patient.

In [6]:
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

### Load partitions by running `MSCK REPAIR TABLE`

As a last step, we need to load the Parquet partitions. To do so, just issue the following SQL command: 

In [7]:
statement = 'MSCK REPAIR TABLE {}.{}'.format(database_name, table_name_parquet)

print(statement)

MSCK REPAIR TABLE dsoaws.amazon_reviews_parquet


In [8]:
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

### Show the partitions

In [9]:
statement = 'SHOW PARTITIONS {}.{}'.format(database_name, table_name_parquet)

print(statement)

SHOW PARTITIONS dsoaws.amazon_reviews_parquet


In [10]:
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

df_partitions = as_pandas(cursor)
df_partitions.head(5)

,partition
0,product_category=Digital_Video_Games
1,product_category=Digital_Software


### Run a sample query

In [11]:
product_category = 'Digital_Software'

statement = """SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100""".format(database_name, table_name_parquet, product_category)

print(statement)

SELECT * FROM dsoaws.amazon_reviews_parquet
    WHERE product_category = 'Digital_Software' LIMIT 100


In [12]:
# Execute statement using connection cursor
cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

df = as_pandas(cursor)
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,year,review_date,product_category
0,US,28523887,R2VX79GL883MWQ,B003O0NP5K,825510099,Serif Digital Scrapbook Artist 2 [Download],2,0,0,N,Y,Okay,It is okay except you have to buy packages to ...,2012,2012-09-06,Digital_Software
1,US,21801173,R3OYNS18EJ1OMZ,B0060C3AG8,376527096,Norton Antivirus 2012 - 1 User,5,0,0,N,Y,"Easy, just look for the small text",It wasn't hard to download and install. The li...,2012,2012-09-06,Digital_Software
2,US,51044431,R1JDRTB2FTURYB,B002UTJGJK,589528089,Math- Algebra II [Download],1,3,4,N,Y,The powerpoint that doesn't pause,This product sucks donkey bowls.<br />Basicall...,2012,2012-09-06,Digital_Software
3,US,48300730,R11547UUVBLNSJ,B003O0NP5K,825510099,Serif Digital Scrapbook Artist 2 [Download],1,3,4,N,N,Do not buy Serif Software,I have purchased several Serif software produc...,2012,2012-09-06,Digital_Software
4,US,13159702,R28CVQRLMLOHNK,B005O24FG4,663353957,Microsoft Access 2010 (2 PC / 1 User) [OLD VER...,2,4,4,N,Y,What's new?,I don't know why Microsoft continues to change...,2012,2012-09-06,Digital_Software


### Done! 

In just a few steps we have set up Amazon Athena to connect to our Amazon Customer Reviews TSV files, and transformed them into Apache Parquet file format. 

You might have noticed that our second sample query finished in a fraction of the time compared to the one before we ran on the TSV table. We speeded up our query results by leveraging our data being stored as Parquet and partitioned by `product_category`. 
